Για την επεξεργασία των αιτήσεων που έχουν ενεργοποιηθεί κατά τη περίοδο 2023-2024 διαδικασία υπολογισμού έχει ως εξής: εντοπίζεται ο μήνας όπου πραγματοποιήθηκε η ενεργοποίηση του έργου, από την ημέρα ενεργοποίησης μέχρι και το τέλος του συγκεκριμένου μήνα εφαρμόζεται ο συντελεστής αξιοποίησης εκείνου του μήνα για την εκτίμηση της παραγόμενης ενέργειας και για τους υπόλοιπους μήνες εφαρμόζονται οι αντίστοιχοι συντελεστές για όλες τις ημέρες του τρέχοντα μήνα κανονικά.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

#Δημιουργία μεταβλητής για το excel που περιέχει τις ενεργοποιημένες αιτήσεις για τα έργα των ενεργειακών κοινοτήτων μαζί με την εγκατεστημένη ισχύς τους
df=pd.read_excel('Between 2023 and 2024.xlsx')

#Δημιουργία μεταβλητής για το excel όπου έχουν γίνει οι υπολογισμοί των μηνιαίων συντελεστών αξιοποίησης για τα αιολικά πάρκα και τα ΦΒ πάρκα της Ηπειροτικής χώρας ξεχωριστά
capacity_factors=pd.read_excel('Capacity Factors.xlsx',sheet_name='Capacity Factors')

Η πλειοψηφία των έργων των ενεργειακών κοινοτήτων αποτελείται από ΦΒ πάρκα. Στο σύνολο των έργων τους υπάρχουν μόνο 2 αιολικά πάρκα (ΑΠ) και 1 σταθμός βιοαερίου. Η παραγωγή ηλεκτρικής ενέργειας του σταθμού βιοαερίου δεν λαμβάνεται υπόψη στους υπολογισμούς αυτής της αναφοράς (ούτως ή άλλως, συγκριτικά με την ενέργεια των ΦΒ είναι αμελητέα).

Το excel με τις ενεργοποιημένες αιτήσεις των ενεργειακών κοινοτήτων περιέχει και τα 2 ΑΠ, οπότε χρειάζονται διαφορετική μεταχείριση από τα ΦΒ στους υπολογισμούς, εφόσον σε αυτά εφαρμόζεται διαφορετικός συντελεστής χωρητικότητας. Το παρακάτω κελί αποθηκεύει τις γραμμές που αφορούν τα 2 ΑΠ και έπειτα τα αφαιρεί από το αρχικό dataframe.

In [2]:
#Αφού έχουν εντοπιστεί οι δείκτες που αντιστοιχούν στα ΑΠ του dataset, δημιουργείται μία λίστα με αυτά
Wind_Parks=[df.iloc[308]['Installed_Capacity_kW'],df.iloc[341]['Installed_Capacity_kW']]

#και στη συνέχεια αφαιρούνται από το αρχικό σετ δεδομένων
df=df.drop(index=[df[df['index']==5454].index[0],df[df['index']==5456].index[0]])

Κατόπιν, δημιουργείται ένα λεξικό με τους αριθμούς μηνών, τις ημέρες του κάθε μήνα και τον μηνιαίο συντελεστή αξιοποίησης για τα ΦΒ πάρκα ως ζεύγη τιμών και ως κλειδιά οι αντίστοιχοι μήνες του έτους και υλοποιούνται τα βήματα που περιγράφηκαν στην αρχή κανονικά, δηλαδή, για τον μήνα ενεργοποίησης, αφαιρείται η ημερομηνία (μέρα) ενεργοποίησης από το σύνολο των ημερών του μήνα ενεργοποίησης του έργου και στη συνέχεια πολλαπλασιάζεται η διαφορά με τον συντελεστή αυτού του μήνα ενώ για τους υπόλοιπους μήνες ο υπολογισμός γίνεται κανονικά.

In [3]:
Months_days={'Jan':(1,31,0.105015),'Feb':(2,28,0.155147),'Mar':(3,31,0.169869),'Apr':(4,30,0.188789),'May':(5,31,0.165171),'Jun':(6,30,0.211803),
        'Jul':(7,31,0.239102),'Aug':(8,31,0.214884),'Sep':(9,30,0.163963),'Oct':(10,31,0.154070),'Nov':(11,30,0.106530),'Dec':(12,31,0.100507)}

#Αυτό το βήμα δεν είναι απαραίτητο μιας και έχουν υπολογιστεί ήδη οι συντελεστές των ΑΠ από το αρχικό excel, αλλά γίνεται για λόγους πληρότητας και εδώ
CF_Wind={}
Energy=[931,958,983,717,850,518,692,1103,1075,778,1156,1030] #GWh Παραγόμενη ενέργεια ανά μήνα
Power=[4465,4465,4504,4522,4519,4692,4705,4723,4743,4828,4895,4948] #MW Εγκατεστημένη ισχύς ανά μήνα (εδώ φαίνεται και ο ρυθμός αύξησης ΑΠ στην Ελλάδα κατά το έτος 2023)
h=[] #κενή λίστα όπου θα χρησιμοποιηθεί για τον υπολογισμό του κλάσματος: ενέργεια/εγκατεστημένη ισχύς


for i in range(len(Energy)):
    h.append(Energy[i]*10**3/Power[i])

#υπολογισμός των μηνιαίων συντελεστών χωρητικότητας για τα ΑΠ
for i,g in zip(Months_days,h):
    CF_Wind[i]=g/(Months_days[i][1]*24)

Energy_Produced=CF_Wind['Nov']*24*30*3000+CF_Wind['Dec']*24*3000*31+CF_Wind['Nov']*24*3000*29+CF_Wind['Dec']*24*3000*31 #kW Υπολογισμός της παραγόμενης ενέργειας από τα 2 ΑΠ
#που ανήκουν σε Εν.Κοιν. 

#Τύπωση της παραγόμενης ενέργειας των ΑΠ από ενεργειακές κοινότητες
print('The energy produced by the 2 wind parks is {} TWh'.format(Energy_Produced*10**-9))

The energy produced by the 2 wind parks is 0.0026423296337063787 TWh


Η διαδικασία υπολογισμού για τα ΦΒ πάρκα που ενεργοποιήθηκαν μεταξύ του 2023 και του 2024 είναι παρόμοια, όμως έγινε με διαφορετικό τρόπο στην python αφού τα δεδομένα για αυτά τα έργα ήταν πολλά περισσότερα.

In [4]:
#Συνάρτηση που αθροίζει τα γινόμενα συντελεστή χωρητικότητας επί τις ημέρες του μήνα για τον οποίο έχει υπολογιστεί ο αντίστοιχος συντελεστής, για όλους τους μήνες του έτους.
#Η συνάρτηση παίρνει ως argument τον μήνα όπου αυτή η διαδικασία παρακάμπτεται (δηλαδή ο μήνας όπου ενεργοποιήθηκε το εκάστοτε έργο). Αφού ο όρος που αφορά την παραγωγή ενέργειας
#από την ημέρα ενεργοποίσης του έργου μέχρι το τέλος του μήνα ενεργοποίησης θα υπολογιστεί πολλαπλασιάζοντας τη διαφορά 
#των ημερών του μήνα ενεργοποίησης από την ημέρα ενεργοποίησης με το συντελεστή του μήνα ενεργοποίησης. 

def equation(month_skip):
    l=0
    for i in Months_days:
        if month_skip<=Months_days[i][0]:
            l+=0
        else:
            l+=Months_days[i][1]*Months_days[i][2]
    return l

#Δημιουργείται κενή λίστα όπου θα καταχωρηθούν τα dataframes που περιέχουν την παραγωγή ενέργειας για τον εκάστοτε μήνα από όλα τα έργα που περιέχονται σε αυτό το excel
dataframes=[]

#Για κάθε μήνα δημιουργείται ξεχωριστό dataframe και εντάσσεται στη κενή λίστα που δημιουργήθηκε από πάνω. Το κάθε dataframe έχει δύο παραπάνω στήλες σε σχέση με το αρχικό:
#μία στήλη που περιέχει τη διαφορά ημερών μήνα από την ημέρα ενεργοποίησης, και μία δεύτερη στήλη που κάνει τους απαραίτητους υπολογισμούς με τη χρήση των συντελεστών χωρητικότητας
#που έχουν υπολογιστεί από το αρχικό excel
for i in Months_days:
    dataframes.append(df[(df['month']==Months_days[i][0])].assign(days_offset_from_month=lambda x: Months_days[i][1]-x['day']).assign(
        Energy_Produced=lambda x: x['days_offset_from_month']*Months_days[i][2]*x['Installed_Capacity_kW']*24+
        x['Installed_Capacity_kW']*equation(Months_days[i][0])*24))
 
#Τα dataframes ενώνονται σε ένα εννιαίο
df_uni=pd.concat(dataframes)

#Τυπώνεται το άθροισμα της παραγώμενης ενέργειας από όλα αυτά τα έργα σε TWh
print("The energy produced by PV parks that were created during the year of 2023 is {} TWh".format(df_uni['Energy_Produced'].sum()*10**-9))

The energy produced by PV parks that were created during the year of 2023 is 0.19354737167854985 TWh


Τα νούμερα που έχουν τυπωθεί από τις γραμμές κώδικα των δύο τελευταίων κελιών καταχωρούνται στο excel που περιέχει τα έργα που έχουν ενεργοποιηθεί πριν το 2023 και η επεξεργασία συνεχίζεται σε αυτό το excel.